In [8]:
import streamlit as st
from streamlit_chat import message
from langchain.chains import ConversationalRetrievalChain
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import CTransformers
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory

In [32]:
pip install ctransformers

     ---------------------------------------- 9.3/9.3 MB 17.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [21]:
#load the pdf files from the path
loader = DirectoryLoader('data/',glob="*.pdf",loader_cls=PyPDFLoader)
documents = loader.load()

In [22]:
#split text into chunks
text_splitter  = RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=50)
text_chunks = text_splitter.split_documents(documents)

In [23]:
#create embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2",
                                   model_kwargs={'device':"cpu"})

In [30]:
#vectorstore
vector_store = FAISS.from_documents(text_chunks,embeddings)

In [35]:
#create llm
llm = CTransformers(model="llama-2-7b-chat.ggmlv3.q4_0.bin",model_type="llama",
                    config={'max_new_tokens':128,'temperature':0.01})

In [36]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [37]:
chain = ConversationalRetrievalChain.from_llm(llm=llm,chain_type='stuff',
                                              retriever=vector_store.as_retriever(search_kwargs={"k":2}),
                                              memory=memory)

In [38]:
st.title("💊HealthCare Assistant🧑🏽‍⚕️")

2023-08-24 17:09:08.768 
  command:

    streamlit run c:\Users\rittw\Documents\Humber\NLP\.conda\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


DeltaGenerator()

In [39]:
def conversation_chat(query):
    result = chain({"question": query, "chat_history": st.session_state['history']})
    st.session_state['history'].append((query, result["answer"]))
    return result["answer"]

In [41]:
def initialize_session_state():
    if 'history' not in st.session_state:
        st.session_state['history'] = []

    if 'generated' not in st.session_state:
        st.session_state['generated'] = ["Hello! Ask me anything about Health 🤗"]

    if 'past' not in st.session_state:
        st.session_state['past'] = ["Hey! 👋"]

In [42]:
def display_chat_history():
    reply_container = st.container()
    container = st.container()

    with container:
        with st.form(key='my_form', clear_on_submit=True):
            user_input = st.text_input("Question:", placeholder="Ask about your Mental Health", key='input')
            submit_button = st.form_submit_button(label='Send')

        if submit_button and user_input:
            output = conversation_chat(user_input)

            st.session_state['past'].append(user_input)
            st.session_state['generated'].append(output)

    if st.session_state['generated']:
        with reply_container:
            for i in range(len(st.session_state['generated'])):
                message(st.session_state["past"][i], is_user=True, key=str(i) + '_user', avatar_style="thumbs")
                message(st.session_state["generated"][i], key=str(i), avatar_style="fun-emoji")


In [43]:
# Initialize session state
initialize_session_state()
# Display chat history
display_chat_history()

2023-08-24 17:17:45.439 Session state does not function when running a script without `streamlit run`


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ c:\Users\rittw\Documents\Humber\NLP\.conda\lib\site-packages\streamlit\runtime\state\session_sta │
│ te.py:378 in __getitem__                                                                         │
│                                                                                                  │
│   375 │   │   │   # the "key" is a raw widget id, so get its associated user key for lookup      │
│   376 │   │   │   key = wid_key_map[widget_id]                                                   │
│   377 │   │   try:                                                                               │
│ ❱ 378 │   │   │   return self._getitem(widget_id, key)                                           │
│   379 │   │   except KeyError:                                                                   │
│   380 │   │   │   raise KeyError(_missing_key_error_message(key))                                │
│   381                                                                                            │
│                                                                                                  │
│ c:\Users\rittw\Documents\Humber\NLP\.conda\lib\site-packages\streamlit\runtime\state\session_sta │
│ te.py:423 in _getitem                                                                            │
│                                                                                                  │
│   420 │   │   │   │   pass                                                                       │
│   421 │   │                                                                                      │
│   422 │   │   # We'll never get here                                                             │
│ ❱ 423 │   │   raise KeyError                                                                     │
│   424 │                                                                                          │
│   425 │   def __setitem__(self, user_key: str, value: Any) -> None:                              │
│   426 │   │   """Set the value of the session_state entry with the given user_key.               │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
KeyError

During handling of the above exception, another exception occurred:

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:4                                                                                    │
│                                                                                                  │
│   1 # Initialize session state                                                                   │
│   2 initialize_session_state()                                                                   │
│   3 # Display chat history                                                                       │
│ ❱ 4 display_chat_history()                                                                       │
│   5                                                                                              │
│                                                                                                  │
│ in display_chat_history:16                                                                       │
│                                                                                                  │
│   13 │   │   │   st.session_state['past'].append(user_input)                                     │
│   14 │   │   │   st.session_state['generated'].append(output)                                    │
│   15 │                                                                                           │
│ ❱ 16 │   if st.session_state['generated']:                                                       │
│   17 │   │   with reply_container:                                               